In [43]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from pandas.plotting import scatter_matrix
from sklearn import svm

In [44]:
datas = pd.read_excel('input/train.xls')

In [45]:
print(datas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 10 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   id                                             1887 non-null   int64 
 1   icerik                                         1887 non-null   object
 2   Gecikti veya Dağıtıma Çıkmadı                  1887 non-null   int64 
 3   Evde yok notu düşüldü veya Kapıya Getirilmedi  1887 non-null   int64 
 4   Telefonlara Cevap Verilmedi                    1887 non-null   int64 
 5   İade Süreci                                    1887 non-null   int64 
 6   Teslim Alınmadı veya Teslim Edilmedi           1887 non-null   int64 
 7   Kötü Diyalog Veya Saygısız Tutum               1887 non-null   int64 
 8   Hasarlı veya Kayıp Paket                       1887 non-null   int64 
 9   Hijyen Kurallarına Uyulmadı                    1887 non-null   

In [46]:
datas['icerik'][:5]

0    Sürat Kargo Gülnur Korkmaz İşitme Engelliyim--...
1    MNG Kargo Teslimat Yapmamak İçin Ellerinden Ge...
2    PTT Kargo Covid Önlemleri Alınmıyor--> PTT Kar...
3    PTT Kargo Ürünü Adrese Getirmeme, Müşteri Hizm...
4    Aras Kargo Kargom Yola Çıkmıyor--> Şubeye gitt...
Name: icerik, dtype: object

In [47]:
X_train, X_test, y_train, y_test = train_test_split(datas.loc[:,'icerik'],datas.drop(['id','icerik'],axis=1), random_state=0)

In [48]:
X_train.shape

(1415,)

In [49]:
y_train.shape

(1415, 8)

In [50]:
X_test.shape

(472,)

In [51]:
y_test.shape

(472, 8)

In [52]:
z=[0,1,2,3,4,5,6,7]
z[0]=datas.loc[:,'Gecikti veya Dağıtıma Çıkmadı']
z[1]=datas.loc[:,'Evde yok notu düşüldü veya Kapıya Getirilmedi']
z[2]=datas.loc[:,'Telefonlara Cevap Verilmedi']
z[3]=datas.loc[:,'İade Süreci']
z[4]=datas.loc[:,'Teslim Alınmadı veya Teslim Edilmedi']
z[5]=datas.loc[:,'Kötü Diyalog Veya Saygısız Tutum']
z[6]=datas.loc[:,'Hasarlı veya Kayıp Paket']
z[7]=datas.loc[:,'Hijyen Kurallarına Uyulmadı']

In [53]:
sikayetsayisi=len(datas)
yontem=TfidfVectorizer

In [54]:
tks = '[A-Za-z0-9]+(?=\\s+)'


pl = Pipeline([
        ('vec', yontem(token_pattern = tks)),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

pl.fit(X_train,y_train)

Pipeline(steps=[('vec', TfidfVectorizer(token_pattern='[A-Za-z0-9]+(?=\\s+)')),
                ('clf', OneVsRestClassifier(estimator=LogisticRegression()))])

In [55]:
test = datas.fillna("")
predictions = pl.predict_proba(test.icerik)

In [56]:
toplam=[0,0,0,0,0,0,0,0]
ortalama=[0,0,0,0,0,0,0,0]
for sutun in range (0,len(z)):
    for i in range (0,len(predictions)):
        toplam[sutun]=toplam[sutun]+predictions[i][sutun]
    ortalama[sutun]=toplam[sutun]/len(predictions)

In [57]:
for i in range (0 , len(predictions)):
    for j in range (0,len(z)):
        if predictions[i][j] < ortalama[j]:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [58]:
# Format predictions in DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=y_test.columns, index=test.id, data=predictions)

prediction_df.to_excel('predictions.xls')

In [59]:
predictions.shape

(1887, 8)

In [60]:
a=X_train
X_train=y_train
y_train=a
a=X_test
X_test=y_test
y_test=a

In [61]:
X_train

,Gecikti veya Dağıtıma Çıkmadı,Evde yok notu düşüldü veya Kapıya Getirilmedi,Telefonlara Cevap Verilmedi,İade Süreci,Teslim Alınmadı veya Teslim Edilmedi,Kötü Diyalog Veya Saygısız Tutum,Hasarlı veya Kayıp Paket,Hijyen Kurallarına Uyulmadı
1728,0,0,1,0,1,0,0,0
987,0,0,0,0,0,1,1,0
133,0,0,0,0,0,1,0,0
202,0,0,0,0,1,1,0,0
1186,1,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...
835,0,1,0,0,0,1,0,0
1216,1,0,0,0,1,0,1,0
1653,0,0,1,0,1,0,1,0
559,1,0,0,0,0,0,0,0


In [62]:
X_test

,Gecikti veya Dağıtıma Çıkmadı,Evde yok notu düşüldü veya Kapıya Getirilmedi,Telefonlara Cevap Verilmedi,İade Süreci,Teslim Alınmadı veya Teslim Edilmedi,Kötü Diyalog Veya Saygısız Tutum,Hasarlı veya Kayıp Paket,Hijyen Kurallarına Uyulmadı
579,1,0,1,0,1,0,0,0
1183,0,0,0,1,1,0,0,0
1010,1,1,0,0,1,0,0,0
1330,1,0,0,0,1,0,0,0
77,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
1546,1,0,1,0,0,0,0,0
936,0,0,0,0,0,1,1,0
632,1,0,1,0,1,0,0,0
1730,1,0,0,0,0,0,0,0


In [63]:
y_train

1728    Aras Kargo Ürünüm Alıcıya Ulaştırılmadı, Geri ...
987     Yurtiçi Kargo Kaybettikleri Kargoyu Israrımızl...
133     PTT Kargo Ağrı Hamur Şubesi Çalışanı--> Geçici...
202     MNG Kargo Ürünümü Teslim Etmiyor--> MNG Kargo ...
1186    Çeliktepe Sürat Kargo'da Sorun Var--> Öncelikl...
                              ...                        
835     Sürat Kargo Kargomu Kapıya Getirmedi Hafif Ola...
1216    Sürat Kargo Teslim Edilmeyen Kargolar--> Trend...
1653    Sürat Kargo İzmir Aktarma Merkezinden Yola Çık...
559     Aras Kargo Kargom 1 Haftadır Gelmiyor--> 1 haf...
684     PTT Kargo Sağlık Çalışanı Olduğum İçin Adrese ...
Name: icerik, Length: 1415, dtype: object

In [64]:
y_test

579     Sürat Kargo İşini Yapmıyor. Telefonları Açmıyo...
1183    Sürat Kargo Yapılmayan Teslimat Sıkıntısı--> M...
1010    PTT Kargo Dağıtılmadan Adreste Yok Bilgisi Gir...
1330    Sürat Kargomun Gecikmesi--> İnternetten aldığı...
77      PTT Kargo Paketi Eve Teslim Etmeme Telefonları...
                              ...                        
1546    Sürat Kargo Kargonun Takip Numarası Gönderilme...
936     Aras Kargo Kayıp Kargom--> 10.07.2020 tarihind...
632     MNG Kargo Adres Yetersiz Gerekçesiyle Mağdur E...
1730    MNG Kargo Teslim Edilemeyen Ürün Sorunu--> Ald...
597     MNG Kargo Kargoyu Dağıtıma Çıkarmama!--> 17 Ağ...
Name: icerik, Length: 472, dtype: object

In [68]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)

In [70]:
clf.score(X_test, y_test)

0.0